In [1]:
from tokenizers import ByteLevelBPETokenizer

In [2]:
tokenizer = ByteLevelBPETokenizer()

In [3]:
!ls *.dedup

chinese-standard.jsonl.dedup
filter-malay-rojak-en.jsonl.dedup
filter-malay-rojak-ms.jsonl.dedup
filter-malay-rojak-rojak.jsonl.dedup
filter-twitter-id.jsonl.dedup
filter-twitter-malay-rojak-id.jsonl.dedup
filter-twitter-malay-rojak-ms.jsonl.dedup
filter-twitter-malay-rojak-rojak.jsonl.dedup
kaskus.jsonl.dedup
local-mandarin.jsonl.dedup
others.jsonl.dedup
others-sample-50k.jsonl.dedup
others-sample.jsonl.dedup
prepare-english-en.jsonl.dedup
prepare-english-ms.jsonl.dedup
prepare-indon.jsonl.dedup
prepare-indon-standard.jsonl.dedup
prepare-malay-ms.jsonl.dedup
prepare-manglish-en.jsonl.dedup
prepare-manglish-manglish.jsonl.dedup
prepare-standard-mandarin.jsonl.dedup
standard-mandarin.jsonl.dedup


In [4]:
import json
from tqdm import tqdm

files = [
    'prepare-malay-ms.jsonl.dedup',
    'prepare-manglish-manglish.jsonl.dedup',
    'prepare-english-ms.jsonl.dedup',
    'kaskus.jsonl.dedup',
    'local-mandarin.jsonl.dedup',
    'standard-mandarin.jsonl.dedup',
    'others-sample.jsonl.dedup'
]

with open('texts', 'w') as fopen_l:
    for f in files:
        print(f)
        count = 0
        with open(f) as fopen:
            for l in tqdm(fopen):
                data = json.loads(l)
                fopen_l.write(f'{data}\n')
                count += 1
                if count >= 1e6:
                    break

prepare-malay-ms.jsonl.dedup


999999it [00:02, 355505.11it/s]


prepare-manglish-manglish.jsonl.dedup


999999it [00:02, 374691.15it/s]


prepare-english-ms.jsonl.dedup


...

999999it [00:05, 176232.96it/s]


standard-mandarin.jsonl.dedup


999999it [00:03, 323622.91it/s]


others-sample.jsonl.dedup


999999it [00:02, 438037.92it/s]


In [3]:
tokenizer.train(
    ['texts'],
    vocab_size=400000,
    min_frequency=2,
    show_progress=True,
    special_tokens=["<s>", "<pad>", "</s>"],
)

In [4]:
tokenizer.save('bpe-400k')

In [2]:
from transformers import AutoTokenizer
from transformers import PreTrainedTokenizerFast

In [3]:
tokenizer = PreTrainedTokenizerFast(tokenizer_file='bpe-400k')

In [7]:
import re

r = re.compile(r'[\S]+').findall

In [8]:
%%time
strings = [
    'tak suka ayam123124123',
    'tak suka ayam'
]
subs = [' '.join(tokenizer.tokenize(s)) for s in strings]
subs[0]

CPU times: user 2.75 ms, sys: 0 ns, total: 2.75 ms
Wall time: 2.47 ms


'tak Ġsuka Ġayam 1 2 3 1 2 4 1 2 3'

In [17]:
vocabulary = list(tokenizer.vocab.keys())

In [18]:
from sklearn.feature_extraction.text import CountVectorizer

bow = CountVectorizer(vocabulary = vocabulary, token_pattern = r'[\S]+').fit(subs)

In [20]:
import pickle

with open('bow.pkl', 'wb') as fopen:
    pickle.dump(bow, fopen)

In [21]:
from sklearn.naive_bayes import ComplementNB

In [26]:
nb = ComplementNB().fit(bow.transform(subs), [0,1])
nb = nb.partial_fit(bow.transform(subs), [0,1])

In [27]:
nb

ComplementNB()

In [28]:
with open('nb.pkl', 'wb') as fopen:
    pickle.dump(nb, fopen)